In [1]:
import jeltz
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.pyplot import cm
import numpy as np
import matplotlib.patches as patches




In [6]:
data = jeltz.marvin('2023ixf')

Looking up TNS info for 2023ixf
Fetching ZTF data through LASAIR
[{'objectId': 'ZTF23aaklqou', 'objectData': {'ncand': 7, 'ramean': 210.9106925857143, 'decmean': 54.3116576, 'glonmean': 101.89171871446808, 'glatmean': 59.768003849860854, 'jdmin': 2460388.9109028, 'jdmax': 2460417.8466204, 'rasex': '14:03:38.566', 'decsex': '54:18:41.967', 'ec_lon': 174.50804512967443, 'ec_lat': 59.812811432022784, 'now_mjd': '60417.61', 'mjdmin_ago': 29.202947627127287, 'mjdmax_ago': 0.26723002719518263, 'discMjd': 60083.32299769996, 'discUtc': '2023-05-19 07:45:07', 'discMag': '15.90±0.03', 'discFilter': 'g', 'latestMjd': 60417.34662039997, 'latestUtc': '2024-04-17 08:19:08', 'latestMag': '17.60±0.04', 'latestFilter': 'g', 'peakMjd': 60118.25777779985, 'peakUtc': '2023-06-23 06:11:12', 'peakMag': '12.02±0.01', 'peakFilter': 'g'}, 'candidates': [{'candid': 2663346622515015004, 'jd': 2460417.8466204, 'ra': 210.9106946, 'dec': 54.311668, 'fid': 1, 'nid': 2663, 'magpsf': 17.599504470825195, 'sigmapsf': 0.

In [7]:

def plot_marvin(tns_name,lightcurve):
    band_color_index = {}
    ignore_telescope_list = []
    order = ['W2', 'UVW2', 'M2', 'UVM2','W1', 'UVW1','U', 'u', 'B','b', 'g', 'c', 'V','v', 'r', 'w', 'R', 'G', 'clear', 'o', 'i', 'I', 'z', 'y', 'J', 'H', 'K','w1','w2']
    offsets = {}
    n = 0
    markersize = 9
    min_offset = int(len(lightcurve.band.unique())/4)

    bands = lightcurve.band.unique()

    bands_for_offset = lightcurve[~lightcurve['telescope'].isin(ignore_telescope_list)].band.unique()
    min_offset = int(len(bands_for_offset)/4)
    fillstyle='full'
    # bands = lightcurve.band.unique()
    for i in order:
        if i in bands_for_offset:
            offsets[i] = min_offset + n
            offsets[i] = 0
            n -= 0.5

    fontsize = 20

    #plt.xkcd()
    # FINAL LC PLOTTER   


    fig, ax = plt.subplots(dpi = 400,figsize = (10,7))

    mStyles = [".","*","d","X","D",">","s","8","p","P","h","H","+","x","X","D","d","|","_"]
    mStyles = mStyles[:len(lightcurve['telescope'].unique())]

    d = {'telescope': lightcurve['telescope'].unique(), 'mstyle': mStyles[:len( lightcurve['telescope'].unique())]}
    markersyles = pd.DataFrame(data=d)

    n_colors = len(lightcurve['band'].unique())
    color_1_def = cm.plasma(np.linspace(0, 0.9,int(round(n_colors+1)/2 )))
    color_2_def = cm.viridis(np.linspace(0.45, 0.9,int(round(n_colors+1)/2 )))

    color_1 = iter(color_1_def)
    color_2 = iter(color_2_def)


    #ax.set_xlim(59700,60250)
    count = 0 
    handles, labels = [] , []


    for filter in order:
        if filter in bands:
            if count < (n_colors / 2):
                c = next(color_1)

            if count >= (n_colors / 2):
                c = next(color_2)

            band_color_index[filter] = c
            count+=1


    #for filter in lightcurve['band'].unique():
    for filter in order:
        if filter in bands:
    
            c = band_color_index[filter]

            single_filter = lightcurve[lightcurve['band']==filter]
            for telescope in lightcurve['telescope'].unique():
                fillstyle='full'
                markersyle = markersyles[markersyles['telescope'] == telescope]['mstyle']
                for_plot = single_filter[single_filter['telescope']==telescope]
                if for_plot.empty:
                    continue

                if telescope in ignore_telescope_list:
                    continue

                offset = offsets[filter]
        
                if telescope == 'synthetic':
                    fillstyle='none'

                print(telescope)
                print(offsets[filter])
                #for_plot_ext = single_filter_ext[single_filter_ext['telescope']==telescope].ext.values
                ax.errorbar(for_plot['time'].astype(float),for_plot['magnitude'].astype(float) + offsets[filter] ,for_plot['e_magnitude'].astype(float),color = c,marker = markersyle.item(), linestyle = '',markersize=markersize,fillstyle=fillstyle)
                fillstyle='full'

            count+=1


    ax.invert_yaxis()

    color_1 = iter(color_1_def)
    color_2 = iter(color_2_def)

    count = 0 

    plt.tight_layout()
    ax.legend()
    ax.set_xlabel(r'MJD',fontsize = fontsize)
    ax.set_ylabel(r'Apparent magnitude',fontsize = fontsize)

    ax.tick_params(axis='both', which='major', labelsize=20)
    ax.tick_params(axis='both', which='minor', labelsize=10)
    ax.minorticks_on()

    ax.set_xlim(np.nanmin(lightcurve['time']) - 10, np.nanmax(lightcurve['time'])+10)
    ax.invert_yaxis()
    ax.invert_yaxis()

    plt.title('MARVIN Output '+ tns_name )


In [8]:

def plot_marvin(lightcurve,ax):
    band_color_index = {}
    band_offset_index = {}

    ignore_telescope_list = []
    order = ['W2', 'UVW2', 'M2', 'UVM2', 'W1', 'UVW1', 'U', 'u', 'B', 'b', 'g', 'c', 'V',
             'v', 'r', 'w', 'R', 'G', 'clear', 'o', 'i', 'I', 'z', 'y', 'J', 'H', 'K', 'w1', 'w2']
    offsets = {}
    n = 0
    markersize = 15
    min_offset = int(len(lightcurve.band.unique())/2)

    bands = lightcurve.band.unique()

    bands_for_offset = lightcurve[~lightcurve['telescope'].isin(
        ignore_telescope_list)].band.unique()
    min_offset = int(len(bands_for_offset)/3)
    fillstyle = 'full'
    # bands = lightcurve.band.unique()
    for i in order:
        if i in bands_for_offset:
            offsets[i] = min_offset + n
            n -= 1

    fontsize = 28

    mStyles = [".", "*", "d", "s", "D", ">", "8", "p",  "X", "P", "h", "H", "+", "x", "X", "D", "d", "|", "_",'.', ',', 'o', 'v', '^', '<', '>', '1', '2', '3', '4', 's', 'p', '*', 'h', 'H', '+', 'x', 'D', 'd', '|', '_']
    
    mStyles = mStyles[:len(lightcurve['telescope'].unique())]

    d = {'telescope': lightcurve['telescope'].unique(
    ), 'mstyle': mStyles[:len(lightcurve['telescope'].unique())]}
    markersyles = pd.DataFrame(data=d)

    n_colors = len(lightcurve['band'].unique())
    color_1_def = cm.plasma(np.linspace(0, 0.9, int(round(n_colors+1)/2)))
    color_2_def = cm.viridis(np.linspace(0.45, 0.9, int(round(n_colors+1)/2)))

    color_1 = iter(color_1_def)
    color_2 = iter(color_2_def)

    # ax.set_xlim(59700,60250)
    count = 0
    handles, labels = [], []

    for filter in order:
        if filter in bands:
            if count < (n_colors / 2):
                c = next(color_1)

            if count >= (n_colors / 2):
                c = next(color_2)

            band_color_index[filter] = c
            count += 1

    for filter in order:
        if filter in bands:

            c = band_color_index[filter]

            count += 1

    # removing limits 
    # actually plotting this time
    # limit_lc = lightcurve[lightcurve['limit'] == 'True']
    # lightcurve = lightcurve[lightcurve['limit'] != 'True']
    # print(limit_lc)

    for filter in order:
        if filter in bands:

            c = band_color_index[filter]
            single_filter = lightcurve[lightcurve['band'] == filter]
            for telescope in lightcurve['telescope'].unique():
                fillstyle = 'full'
                markersyle = markersyles[markersyles['telescope']
                                         == telescope]['mstyle']
                for_plot = single_filter[single_filter['telescope']
                                         == telescope]
                if for_plot.empty:
                    continue

                if telescope in ignore_telescope_list:
                    continue

                offset = offsets[filter]
                band_offset_index[filter] = offset

                if telescope == 'synthetic':
                    fillstyle = 'none'

                ax.errorbar(for_plot['time'].astype(float), for_plot['magnitude'].astype(float) + offsets[filter], for_plot['e_magnitude'].astype(
                    float), color=c, marker=markersyle.item(), linestyle='', markersize=markersize, fillstyle=fillstyle)
                fillstyle = 'full'

            count += 1

    # plotting up the limits
    # if len(limit_lc) > 0: 
    #     for filter in order:
    #         if filter in bands:

    #             c = band_color_index[filter]
    #             single_filter = limit_lc[limit_lc['band'] == filter]
    #             for telescope in limit_lc['telescope'].unique():
    #                 fillstyle = 'full'
    #                 markersyle = markersyles[markersyles['telescope']
    #                                         == telescope]['mstyle']
    #                 for_plot = single_filter[single_filter['telescope']
    #                                         == telescope]
    #                 if for_plot.empty:
    #                     continue

    #                 if telescope in ignore_telescope_list:
    #                     continue

    #                 offset = offsets[filter]
    #                 band_offset_index[filter] = offset

    #                 if telescope == 'synthetic':
    #                     fillstyle = 'none'

    #                 ax.plot(for_plot['time'].astype(float), for_plot['magnitude'].astype(float) + offsets[filter], color=c, marker=markersyle.item(), linestyle='', markersize=markersize, fillstyle='none', markeredgewidth=2.5)


    ax.invert_yaxis()

    color_1 = iter(color_1_def)
    color_2 = iter(color_2_def)

    count = 0

    for i in order:
        if i in bands_for_offset:

            c = band_color_index[i]

            offset = offsets[i]
            if offset < 0:
                offset_str = ''
            else:
                offset_str = '+'
            patch = ax.add_patch(patches.Rectangle(
                (0, 0), 0, 0, facecolor=c, label=i+' '+offset_str+str(offsets[i])))
            count += 1

    handles, labels = ax.get_legend_handles_labels()
    handle_list, label_list = [], []
    for handle, label in zip(handles, labels):
        if label not in label_list:
            handle_list.append(handle)
            label_list.append(label)

    # bands
    legend1 = plt.legend(handles=handle_list, ncol=2, prop={
              'size': 14}, fontsize=30, title_fontsize=16, markerscale=2., loc='upper center', bbox_to_anchor=(0.74, 2.5), fancybox=True, shadow=False,)

    # point syle
    handle_list, label_list = [], []
    for telescope in lightcurve['telescope'].unique():
        if telescope in ignore_telescope_list:
            continue
        markersyle = markersyles[markersyles['telescope']
                                 == telescope]['mstyle']
        markersyle = markersyle.item()
        handle = ax.errorbar([], [], y_err=None, color='grey', marker=markersyle,
                             linestyle='', label=telescope, fillstyle='none')
        handle_list.append(handle)
    #plt.tight_layout()

    ax.set_xlabel(r'MJD', fontsize=fontsize)
    ax.set_ylabel(r'Apparent magnitude + offset', fontsize=fontsize)

    ax.minorticks_on()
    ax.set_xlim(np.nanmin(lightcurve['time']) -
                10, np.nanmax(lightcurve['time'])+10)
                
    ax.invert_yaxis()
    ax.invert_yaxis()
    # telescopes
    legend2 = plt.legend(handles=handle_list, ncol=2, prop={
               'size': 14}, fontsize=30, title_fontsize=16, markerscale=2., loc='upper center', bbox_to_anchor=(0.74, 6.5), fancybox=True, shadow=False,frameon=False)
    plt.gca().add_artist(legend1)
    plt.gca().add_artist(legend2)

    ax.tick_params(axis='both', which='major', labelsize=26)
    ax.tick_params(axis='both', which='minor', labelsize=20)
    return band_color_index, band_offset_index

In [9]:
f, ax = plt.subplots(nrows=1, ncols=1, sharex=True,dpi = 700,figsize = (12,12))
plot_marvin(data,ax)
plt.legend(frameon=False, markerscale=2.2,prop={ 'size': 19}, ncols = 2)
